In [1]:
#from datasets import IRSPRT#RecgArtPrintNoIntsectHVBW
import pytesseract as pyt
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse
from glob import glob
import cv2
import numpy as np
import os
import time
from PIL import Image
import editdistance

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
#from datasets import IRS #ArtPrintNoIntsectLBW,ArtPrintNoIntsectLBW_biameyd_siameyd,ArtPrintNoIntsectLBW_bpr_spr,ArtPrintNoIntsectLBW_biameyd_sprt
############from Model_Unet_github import *
from utils_seg import *
import utils_recg


from Model_Binary_Joint import *
############from recognition.Model import RecgModel, DecoderType
#from recognition.utils import log_image

home = os.environ['HOME']

In [2]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
#parser.add_argument("-name", default='segnet_unet_hvbw_all_combine_100_epoches', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-name_seg", default='new_segnet_joint', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
#segnet_binary_100epoch_unet_github
parser.add_argument("-gpu", default='-1', type=str, help="gpu numbers")

parser.add_argument("-train", default=False, help="train the NN", action="store_true")
parser.add_argument("-validate", help="validate the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=True, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size_seg", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=100, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/root/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/root/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()



_StoreAction(option_strings=['-urlTranferFrom'], dest='urlTranferFrom', nargs=None, const=None, default='', type=<class 'str'>, choices=None, help=' archived model url ', metavar=None)

In [3]:
# Recognition Model
# basic operations
parser.add_argument("-name_recg", default="recg_new_five_datasets", type=str, help="name of the log") #'dense_128_32_noartifact_beamsearch_5_datasets'
#parser.add_argument("-gpu", default='-1', type=str, help="gpu numbers")
#parser.add_argument("-train", help="train the NN", action="store_true")
#parser.add_argument("-validate", help="validate the NN", action="store_true")
#parser.add_argument("-transfer", action="store_true")
#actually not effective:
parser.add_argument("-batchesTrained", default=0, type=int, help='number of batches already trained (for lr schedule)') 
# beam search
parser.add_argument("-beamsearch", help="use beam search instead of best path decoding",default=True, action="store_true")
parser.add_argument("-wordbeamsearch", help="use word beam search instead of best path decoding", action="store_true")
# training hyperparam
parser.add_argument("-batchsize_recg", default=10, type=int, help='batch size') # actually not effective in infrerence
#parser.add_argument("-lrInit", default=1e-2, type=float, help='initial learning rate') # actually not effective
parser.add_argument("-optimizer", default='rmsprop', help="adam, rmsprop, momentum") # actually not effective
parser.add_argument("-wdec", default=1e-4, type=float, help='weight decay') # acctually not effective
#parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time')
#parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time')
parser.add_argument("-epochEnd", default=40, type=int, help='end after this many epochs')
# trainset hyperparam
#parser.add_argument("-noncustom", help="noncustom (original) augmentation technique", action="store_true")
#parser.add_argument("-noartifact", help="dont insert artifcats", action="store_true")
#parser.add_argument("-iam", help='use iam dataset', action='store_true')
# densenet hyperparam
parser.add_argument("-nondensenet", help="use noncustom (original) vanilla cnn", action="store_true")
parser.add_argument("-growth_rate", default=12, type=int, help='growth rate (k)')
parser.add_argument("-layers_per_block", default=18, type=int, help='number of layers per block')
parser.add_argument("-total_blocks", default=5, type=int, help='nuber of densenet blocks')
parser.add_argument("-keep_prob", default=1, type=float, help='keep probability in dropout')
parser.add_argument("-reduction", default=0.4, type=float, help='reduction factor in 1x1 conv in transition layers')
parser.add_argument("-bc_mode", default=True, type=bool, help="bottleneck and compresssion mode")
# rnn,  hyperparams
parser.add_argument("-rnndim", default=256, type=int, help='rnn dimenstionality') #256
parser.add_argument("-rnnsteps", default=32, type=int, help='number of desired time steps (image slices) to feed rnn')
# img size
parser.add_argument("-imgsize", default=[128,32], type=int, nargs='+') #qyk default 128,32 // use segnet definition
# testset crop
#parser.add_argument("-crop_r1", default=3, type=int)
#parser.add_argument("-crop_r2", default=28, type=int)
#parser.add_argument("-crop_c1", default=10, type=int)
#parser.add_argument("-crop_c2", default=115, type=int)
# filepaths
parser.add_argument("-dataroot", default='/root/datasets', type=str)
#######parser.add_argument("-ckptroot", default='/root/ckpt', type=str)##############
#parser.add_argument("-urlTransferFrom", default=None, type=str)

args = parser.parse_known_args()[0]

### SegNet
home = os.environ['HOME']
#name = args.name
#ckptroot = join(home, 'ckpt')
#args.ckptpath = join(ckptroot, name)
#os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

####args = parser.parse_known_args()[0]

name_seg = args.name_seg
name_recg=args.name_recg

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath_seg = join(ckptroot, name_seg)
args.ckptpath_recg = join(ckptroot, name_recg)
if args.name_seg=='debug_seg_': shutil.rmtree(args.ckptpath_seg, ignore_errors=True)
if args.name_recg=='debug_recg_': shutil.rmtree(args.ckptpath_recg, ignore_errors=True)

os.makedirs(args.ckptpath_seg, exist_ok=True)
os.makedirs(args.ckptpath_recg, exist_ok=True)

#recg_name=args.recg_name
#args.regckptpath=join(ckptroot,recg_name)

In [4]:
from datasets_seg import *

# IRS

In [5]:
transform_train = transforms.Compose([
    transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
    transforms.Lambda(lambda img: np.expand_dims(img,3) ),
    #transforms.Lambda(lambda img: add_artifacts(img,args)),
    #transforms.Lambda(lambda img: cv2.transpose(img))
    ])
testset=IRSPRT(transform=transform_train)
testloader = DataLoader(testset, batch_size=args.batch_size_seg, shuffle=False, drop_last=False,num_workers=2)

In [20]:
numWordsIRS=len(testset)
numWordsIRS

22165

In [33]:
upload=True
#numCharErr, numCharTotal, numWordOK, numWordTotal = 0, 0, 0, 0
#numCharErrClean, numWordOKClean = 0, 0
numCharTotal=0
#kernel = np.ones((2,2), np.uint8)
#plt.figure(figsize=(6,2))
#counter = 0
lenidx=len(testset)
for idx, (images,labels) in enumerate(testloader):
    #if np.mod(idx,100)==0:
    #  print(str(idx/lenidx))
    #images=images.numpy()
    #print(len(labels))
    for label in labels:
        #print(label)
        numCharTotal+=len(label)
print(numCharTotal)
print(args.batch_size_seg)
lastbatchsizeIRS=len(labels)
print(lastbatchsizeIRS)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


70596
10
5


In [34]:
mutiple_IRS=((numWordsIRS-lastbatchsizeIRS)*args.batch_size_seg+lastbatchsizeIRS**2)/numCharTotal
mutiple_IRS

3.13934217236104

# Complete PRT size batch_segx 5

In [45]:
transform_train = transforms.Compose([
    transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
    transforms.Lambda(lambda img: np.expand_dims(img,3) ),
    #transforms.Lambda(lambda img: add_artifacts(img,args)),
    #transforms.Lambda(lambda img: cv2.transpose(img))
    ])
testset=ArtPrintNoIntsectBinary(transform=transform_train)
testloader = DataLoader(testset, batch_size=args.batch_size_seg*5, shuffle=False, drop_last=False,num_workers=2)

/root/datasets/artifact_images_no_intersect already exists, skipping download


In [44]:
numWordsPRT=len(testset)
numWordsPRT

284246

In [49]:
upload=True
#numCharErr, numCharTotal, numWordOK, numWordTotal = 0, 0, 0, 0
#numCharErrClean, numWordOKClean = 0, 0
numCharTotal=0
#kernel = np.ones((2,2), np.uint8)
#plt.figure(figsize=(6,2))
#counter = 0
lenidx=len(testset)
for idx, (images,_,labels) in enumerate(testloader):
    #if np.mod(idx,100)==0:
    #  print(str(idx/lenidx))
    #images=images.numpy()
    #print(len(labels))
    for label in labels:
        #print(label)
        numCharTotal+=len(label)
print(numCharTotal)
print(args.batch_size_seg)
lastbatchsizePRTx5=len(labels)
print(lastbatchsizePRTx5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


902089
10
46


In [50]:
mutiple_PRTx5=((numWordsPRT-lastbatchsizePRTx5)*args.batch_size_seg*5+lastbatchsizePRTx5**2)/numCharTotal
mutiple_PRTx5

15.754671656566037

# Complete PRT size batch_seg

In [39]:
transform_train = transforms.Compose([
    transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
    transforms.Lambda(lambda img: np.expand_dims(img,3) ),
    #transforms.Lambda(lambda img: add_artifacts(img,args)),
    #transforms.Lambda(lambda img: cv2.transpose(img))
    ])
testset=ArtPrintNoIntsectBinary(transform=transform_train)
testloader = DataLoader(testset, batch_size=args.batch_size_seg, shuffle=False, drop_last=False,num_workers=2)

/root/datasets/artifact_images_no_intersect already exists, skipping download


In [47]:
numWordsPRT=len(testset)
numWordsPRT

284246

In [41]:
upload=True
#numCharErr, numCharTotal, numWordOK, numWordTotal = 0, 0, 0, 0
#numCharErrClean, numWordOKClean = 0, 0
numCharTotal=0
#kernel = np.ones((2,2), np.uint8)
#plt.figure(figsize=(6,2))
#counter = 0
lenidx=len(testset)
for idx, (images,_,labels) in enumerate(testloader):
    #if np.mod(idx,100)==0:
    #  print(str(idx/lenidx))
    #images=images.numpy()
    #print(len(labels))
    for label in labels:
        #print(label)
        numCharTotal+=len(label)
print(numCharTotal)
print(args.batch_size_seg)
lastbatchsizePRT=len(labels)
print(lastbatchsizePRT)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


902089
10
6


In [51]:
mutiple_PRT=((numWordsPRT-lastbatchsizePRT)*args.batch_size_seg+lastbatchsizePRT**2)/numCharTotal
mutiple_PRT

3.1509485206005174

# 20000 PRT size batch_seg

In [52]:
transform_train = transforms.Compose([
    transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
    transforms.Lambda(lambda img: np.expand_dims(img,3) ),
    #transforms.Lambda(lambda img: add_artifacts(img,args)),
    #transforms.Lambda(lambda img: cv2.transpose(img))
    ])
testset=ArtPrintNoIntsectBinary_20000(transform=transform_train)
testloader = DataLoader(testset, batch_size=args.batch_size_seg, shuffle=False, drop_last=False,num_workers=2)

/root/datasets/artifact_images_no_intersect already exists, skipping download


In [53]:
numWordsPRT_20000=len(testset)
numWordsPRT_20000

20000

In [54]:
upload=True
#numCharErr, numCharTotal, numWordOK, numWordTotal = 0, 0, 0, 0
#numCharErrClean, numWordOKClean = 0, 0
numCharTotal=0
#kernel = np.ones((2,2), np.uint8)
#plt.figure(figsize=(6,2))
#counter = 0
lenidx=len(testset)
for idx, (images,_,labels) in enumerate(testloader):
    #if np.mod(idx,100)==0:
    #  print(str(idx/lenidx))
    #images=images.numpy()
    #print(len(labels))
    for label in labels:
        #print(label)
        numCharTotal+=len(label)
print(numCharTotal)
print(args.batch_size_seg)
lastbatchsizePRT_20000=len(labels)
print(lastbatchsizePRT_20000)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


63750
10
10


In [55]:
mutiple_PRT_20000=((numWordsPRT_20000-lastbatchsizePRT_20000)*args.batch_size_seg+lastbatchsizePRT_20000**2)/numCharTotal
mutiple_PRT_20000

3.1372549019607843

# Complete HBVW

In [56]:
transform_train = transforms.Compose([
    transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
    transforms.Lambda(lambda img: np.expand_dims(img,3) ),
    #transforms.Lambda(lambda img: add_artifacts(img,args)),
    #transforms.Lambda(lambda img: cv2.transpose(img))
    ])
testset=RecgArtPrintNoIntsectHVBW(transform=transform_train)
testloader = DataLoader(testset, batch_size=args.batch_size_seg, shuffle=False, drop_last=False,num_workers=2)

In [57]:
numWordsHBVW=len(testset)
numWordsHBVW

284241

In [58]:
upload=True
#numCharErr, numCharTotal, numWordOK, numWordTotal = 0, 0, 0, 0
#numCharErrClean, numWordOKClean = 0, 0
numCharTotal=0
#kernel = np.ones((2,2), np.uint8)
#plt.figure(figsize=(6,2))
#counter = 0
lenidx=len(testset)
for idx, (images,labels) in enumerate(testloader):
    #if np.mod(idx,100)==0:
    #  print(str(idx/lenidx))
    #images=images.numpy()
    #print(len(labels))
    for label in labels:
        #print(label)
        numCharTotal+=len(label)
print(numCharTotal)
print(args.batch_size_seg)
lastbatchsizeHBVW=len(labels)
print(lastbatchsizeHBVW)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


902076
10
1


In [59]:
mutiple_HBVW=((numWordsHBVW-lastbatchsizeHBVW)*args.batch_size_seg+lastbatchsizeHBVW**2)/numCharTotal
mutiple_HBVW

3.150955130166416